In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Modeling
from sklearn.linear_model import LinearRegression

# Preprocessing - Data standardization
from sklearn.preprocessing import StandardScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

<IPython.core.display.Javascript object>

# Functions and definitions

## Helper functions for blocked time series cross validation

### Train test split

In [3]:
def split_by_periods(x, y, train_period, test_period):
    datasets = []
    i = 0
    max_samples = x.shape[0]

    for _ in range(0, max_samples // (train_period + test_period)):
        # Splitting the data into train/test sets
        x_train = x[i : i + train_period].copy()
        y_train = y[i : i + train_period].copy()
        x_test = x[i + train_period : i + train_period + test_period].copy()
        y_test = y[i + train_period : i + train_period + test_period].copy()

        datasets.append(
            {
                "x_train": x_train,
                "y_train": y_train,
                "x_test": x_test,
                "y_test": y_test,
            }
        )
        # Increments the index for the next period of time
        i += train_period + test_period

    return datasets

<IPython.core.display.Javascript object>

In [4]:
def split_by_dates(x, y, train_period, test_period, dates):
    datasets = []
    dates = dates[: x.shape[0]].copy()
    dates_unique = dates.copy().unique()
    i = 0
    max_samples = dates_unique.shape[0]

    for _ in range(0, max_samples // (train_period + test_period)):
        # Splitting the Train Set
        start_date_train = dates_unique[i]
        end_date_train = dates_unique[i + train_period]
        idx_train = dates[(dates >= start_date_train) & (dates < end_date_train)].index
        x_train = x.loc[idx_train].copy()
        y_train = y[idx_train].copy()

        # Splitting the Test Set
        start_date_test = dates_unique[i + train_period]
        end_date_test = dates_unique[i + train_period + test_period]
        idx_test = dates[(dates >= start_date_test) & (dates < end_date_test)].index
        x_test = x.loc[idx_test].copy()
        y_test = y[idx_test].copy()

        datasets.append(
            {
                "x_train": x_train,
                "y_train": y_train,
                "x_test": x_test,
                "y_test": y_test,
            }
        )

        i += train_period + test_period

    return datasets

<IPython.core.display.Javascript object>

In [5]:
def train_test_split_blocked_ts(x, y, train_period, test_period, dates=None):
    """
    Split the input data into train-test datasets based on train and test periods.

    Args:
        x (pd.DataFrame): Input features.
        y (np.Array): Target values.
        train_period (int): Length of the training period.
        test_period (int): Length of the testing period.
        dates (pd.Series): Optional date information.

    Returns:
        List[dict]: A list of dictionaries, each containing 'x_train', 'y_train', 'x_test', and 'y_test'.
    """
    if dates is None:
        return split_by_periods(x, y, train_period, test_period)
    else:
        return split_by_dates(x, y, train_period, test_period, dates)

<IPython.core.display.Javascript object>

### Data preprocessing

In [6]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [7]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [8]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [9]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

### Blocking time series cross validation

In [10]:
def repeated_blocking_time_series(
    Estimator,
    Transform,
    Imputer,
    x,
    y,
    train_period,
    test_period,
    dates=None,
    repeats=10,
    estimator_params=None,
    imputer_params=None,
):
    """
    Perform repeated cross-validation with blocked time series data.

    Args:
        Estimator: Machine learning model class.
        Transform: Data transformation method.
        Imputer: Data imputation method.
        x: Input features.
        y: Target values.
        train_period: Length of the training period.
        test_period: Length of the testing period.
        dates: Optional date information.
        repeats: Number of repetitions.
        estimator_params: Parameters for the model.
        imputer_params: Parameters for data imputation.

    Returns:
        list: List of dictionaries containing evaluation metrics for each repetition.
    """

    results = []
    max_samples = x.shape[0]

    # Splitting the data into train/test sets
    datasets = train_test_split_blocked_ts(x, y, train_period, test_period, dates)

    for _ in range(repeats):
        scores = []

        for dataset in datasets:
            dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)
            score = train_and_evaluate_model(Estimator, dataset, estimator_params)
            scores.append(score)

        # After every iteration metrics results are appended together
        scores_final = {key: [] for key, _ in scores[0].items()}
        for scores_dict in scores:
            for key, value in scores_dict.items():
                scores_final[key] += [value]
        results.append(scores_final)
    return results

<IPython.core.display.Javascript object>

In [11]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}
CEM_TYPE = ["cem_type_CEM B", "cem_type_CEM C"]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [12]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_iv",
    "Features": "Chemical + Mineralogical + Physical",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [13]:
df = pd.read_csv("../../../../../data/processed/partner_iv/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

In [14]:
df_copy = df.copy().drop(CEM_TYPE, axis=1)

<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_copy</h2> <br>In this dataset all features are used.

In [15]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 37)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>5<br>
    1. 5 folds of 246 samples each
    2. 80% train (988 samples each fold)
    3. 20% test (246 samples each fold)
<b>Total:</b> 15 models<br>

In [16]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.171 (0.020)
MAE: -1.688 (0.018)
MAPE: -0.035 (0.000)
R2: 0.892 (0.002)


******
[TEST]
******
RMSE: -2.289 (0.099)
MAE: -1.773 (0.067)
MAPE: -0.037 (0.001)
R2: 0.879 (0.010)




<IPython.core.display.Javascript object>

In [17]:
results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

<IPython.core.display.Javascript object>

In [18]:
df_results.groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(["mean", "std"]).reset_index()

Features              Model Cross Validation  \
                                                                             
0  Chemical + Mineralogical + Physical  Linear Regression   Repeated KFold   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  2.289091  0.102427  1.773385  0.069146  0.036504  0.001544  0.879376   

            
       std  
0  0.01015

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 80% train (987 samples each fold)
    3. 20% test (246 samples each fold)
<b>Total:</b> 5 models<br>

In [19]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.688 (0.170)
MAE: -1.349 (0.141)
MAPE: -0.028 (0.003)
R2: 0.932 (0.015)


******
[TEST]
******
RMSE: -2.809 (0.850)
MAE: -2.241 (0.611)
MAPE: -0.046 (0.013)
R2: 0.766 (0.156)




<IPython.core.display.Javascript object>

In [20]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                            
0  Chemical + Mineralogical + Physical  Linear Regression   
1  Chemical + Mineralogical + Physical  Linear Regression   

             Cross Validation Timesteps RMSE Test            MAE Test  \
                                             mean       std      mean   
0  Blocking Time Series Split       NaN  2.808859  0.850106  2.240956   
1              Repeated KFold       NaN  2.289091  0.098954  1.773385   

            MAPE Test             R2 Test            
        std      mean       std      mean       std  
0  0.611031  0.045767  0.012924  0.765630  0.155857  
1  0.066801  0.036504  0.001492  0.879376  0.009806

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [21]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.076 (0.067)
MAE: -1.630 (0.029)
MAPE: -0.034 (0.001)
R2: 0.899 (0.008)


******
[TEST]
******
RMSE: -3.376 (1.412)
MAE: -2.609 (1.122)
MAPE: -0.054 (0.024)
R2: 0.664 (0.339)




<IPython.core.display.Javascript object>

In [22]:
results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

<IPython.core.display.Javascript object>

In [23]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                            
0  Chemical + Mineralogical + Physical  Linear Regression   
1  Chemical + Mineralogical + Physical  Linear Regression   
2  Chemical + Mineralogical + Physical  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  2.808859  0.850106  2.240956  0.611031   
1              Repeated KFold  2.289091  0.098954  1.773385  0.066801   
2           Time Series Split  3.375874  1.412288  2.608966  1.121683   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.045767  0.012924  0.765630  0.155857  
1  0.036504  0.001492  0.879376  0.009806  
2  0.053924  0.024405  0.663683  0.338850

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [24]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

******
[TRAIN]
******
RMSE: 2.126 (0.000)
MAE: 1.658 (0.000)
MAPE: 0.034 (0.000)
R2: 0.899 (0.000)


******
[TEST]
******
RMSE: 2.678 (0.000)
MAE: 2.126 (0.000)
MAPE: 0.044 (0.000)
R2: 0.817 (0.000)




<IPython.core.display.Javascript object>

In [25]:
results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

<IPython.core.display.Javascript object>

In [26]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                            
0  Chemical + Mineralogical + Physical  Linear Regression   
1  Chemical + Mineralogical + Physical  Linear Regression   
2  Chemical + Mineralogical + Physical  Linear Regression   
3  Chemical + Mineralogical + Physical  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  2.808859  0.850106  2.240956  0.611031   
1           Out of time Split  2.678265  0.000000  2.126159  0.000000   
2              Repeated KFold  2.289091  0.098954  1.773385  0.066801   
3           Time Series Split  3.375874  1.412288  2.608966  1.121683   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.045767  0.012924  0.765630  0.155857  
1  0.043671  0.000000  0.816640  0.000000  
2  0.036504  0.001492  0.879376  0.009806  
3  0.053924  0.024405  0.663683  0.338850

<IPython.core.display.Javascript object>

In [27]:
pipeline.named_steps["estimator"].coef_

array([-1.51760149e-01, -1.23920435e+00, -2.91401405e-01, -5.13682271e-02,
        6.85737426e-02,  1.81981312e-01,  2.74469705e-01,  3.14569724e+00,
       -4.24234376e-01,  2.63534631e-01, -8.87672883e+00, -4.72946192e+00,
        8.01460461e+01, -1.36424681e+00, -3.28277985e-02,  2.21100526e+01,
       -6.27453285e-01, -1.08008343e+00,  8.73319595e+00,  1.49181018e+01,
        2.58470709e+00,  3.62242392e+00,  4.43161119e+00,  1.50541502e+00,
        4.53653794e+00,  8.13597887e-01,  7.14839134e-01,  4.94567003e-01,
        3.90674889e+00,  1.62932123e+00,  2.21512598e+00,  8.19979910e+01,
        4.75321505e+00, -1.42700571e+00])

<IPython.core.display.Javascript object>

In [28]:
pipeline.named_steps["estimator"].intercept_

49.22584856396868

<IPython.core.display.Javascript object>

In [29]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [30]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['SO3', 'Cl-', 'Blaine', 'ph2oimm', 'Initial Setting Time', 'Soundness',
       'flow', 'CS2', 'residuo 24 micron', 'R_wp', 'Alite_M3 C3S M3',
       'Alite_M1 C3S M1', 'Alite_Sum C3S tot', 'Ratio_M1 (rapporto M1/M3)',
       'C3S_CS (taglia dei cristalliti C3S)', 'Belite_beta', 'C3A cub',
       'C3A_ortho', 'C3A tot', 'C4AF', 'CaO', 'Ca(OH)2', 'Calce libera',
       'Periclasio (MgO)', 'Quartz', 'K2SO4', 'Langbeinite – MgK2(SO4)2',
       'Aphthitalite – (K,Na)3(SO4)2', 'Gesso', 'Emiidrato', 'Anidrite',
       'Calcite – CaCO3', 'SO3_XRD', 'CO2_XRD'],
      dtype='object')

<IPython.core.display.Javascript object>

In [31]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(34,)

<IPython.core.display.Javascript object>

In [32]:
x.columns.shape

(34,)

<IPython.core.display.Javascript object>

In [33]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [34]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
Calcite – CaCO3,81.997991
Alite_Sum C3S tot,80.146046
Belite_beta,22.110053
C4AF,14.918102
C3A tot,8.733196
SO3_XRD,4.753215
Quartz,4.536538
Calce libera,4.431611
Gesso,3.906749
Ca(OH)2,3.622424


<IPython.core.display.Javascript object>

In [35]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                            
0  Chemical + Mineralogical + Physical  Linear Regression   
1  Chemical + Mineralogical + Physical  Linear Regression   
2  Chemical + Mineralogical + Physical  Linear Regression   
3  Chemical + Mineralogical + Physical  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  2.808859  0.850106   
1                                {"Test Size": 0.2}  2.678265  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  2.289091  0.098954   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  3.375874  1.412288   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  2.240956  0.611031  0.045767  0.012924  0.765630  0.155857  
1  2.126159  0.000000  0.043671  0.000000  0.816640  0.000000  
2  1.773385  0.066801  0.036504  0.001492  0.879376  0.009806  
3  2.608966  1.121683  0.053924  0.024405  0.663683  0.338850

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [36]:
path = "../../../../../reports/results/local_models/partner_iv/all_cements/full/"
filename = "linear_regression_results_full_5.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [37]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_iv/all_cements/grouped/"
filename = "linear_regression_results_grouped_5.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [38]:
df_results_to_save

Category Company Data Shape Timesteps  \
                                              
0  Local Model   partner_iv  (958, 34)       NaN   
1  Local Model   partner_iv  (958, 34)       NaN   
2  Local Model   partner_iv  (958, 34)       NaN   
3  Local Model   partner_iv  (958, 34)       NaN   

                              Features              Model  \
                                                            
0  Chemical + Mineralogical + Physical  Linear Regression   
1  Chemical + Mineralogical + Physical  Linear Regression   
2  Chemical + Mineralogical + Physical  Linear Regression   
3  Chemical + Mineralogical + Physical  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   1.688344  0.169859  ...   
1                                {"Test Size": 0.2}   2.125734  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   2.171341  0.019960  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   2.076351  0.066849  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.932211  0.014818  2.808859  0.850106  2.240956  0.611031  0.045767   
1  0.899006  0.000000  2.678265  0.000000  2.126159  0.000000  0.043671   
2  0.892184  0.002209  2.289091  0.098954  1.773385  0.066801  0.036504   
3  0.899291  0.007667  3.375874  1.412288  2.608966  1.121683  0.053924   

              R2 Test            
        std      mean       std  
0  0.012924  0.765630  0.155857  
1  0.000000  0.816640  0.000000  
2  0.001492  0.879376  0.009806  
3  0.024405  0.663683  0.338850  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>

In [39]:
c = pd.read_csv(
    "../../../../../reports/results/local_models/partner_i-oficial/all_cements/grouped/linear_regression_results_grouped_1.csv",
    header=[0, 1],
).rename(columns=lambda x: "" if "Unnamed" in x else x, level=1)

<IPython.core.display.Javascript object>

In [40]:
c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   (Category, )                 4 non-null      object 
 1   (Company, )                  4 non-null      object 
 2   (Data Shape, )               4 non-null      object 
 3   (Timesteps, )                0 non-null      float64
 4   (Features, )                 4 non-null      object 
 5   (Model, )                    4 non-null      object 
 6   (Cross Validation, )         4 non-null      object 
 7   (Cross Validation Params, )  4 non-null      object 
 8   (RMSE Train, mean)           4 non-null      float64
 9   (RMSE Train, std)            4 non-null      float64
 10  (MAE Train, mean)            4 non-null      float64
 11  (MAE Train, std)             4 non-null      float64
 12  (MAPE Train, mean)           4 non-null      float64
 13  (MAPE Train, std)       

<IPython.core.display.Javascript object>

In [41]:
df_results_to_save.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   (Category, )                 4 non-null      object 
 1   (Company, )                  4 non-null      object 
 2   (Data Shape, )               4 non-null      object 
 3   (Timesteps, )                0 non-null      float64
 4   (Features, )                 4 non-null      object 
 5   (Model, )                    4 non-null      object 
 6   (Cross Validation, )         4 non-null      object 
 7   (Cross Validation Params, )  4 non-null      object 
 8   (RMSE Train, mean)           4 non-null      float64
 9   (RMSE Train, std)            4 non-null      float64
 10  (MAE Train, mean)            4 non-null      float64
 11  (MAE Train, std)             4 non-null      float64
 12  (MAPE Train, mean)           4 non-null      float64
 13  (MAPE Train, std)       

<IPython.core.display.Javascript object>

In [42]:
df_results_to_save[df_results_to_save["Cross Validation Params"].str.contains("Date")]

Empty DataFrame
Columns: [(Category, ), (Company, ), (Data Shape, ), (Timesteps, ), (Features, ), (Model, ), (Cross Validation, ), (Cross Validation Params, ), (RMSE Train, mean), (RMSE Train, std), (MAE Train, mean), (MAE Train, std), (MAPE Train, mean), (MAPE Train, std), (R2 Train, mean), (R2 Train, std), (RMSE Test, mean), (RMSE Test, std), (MAE Test, mean), (MAE Test, std), (MAPE Test, mean), (MAPE Test, std), (R2 Test, mean), (R2 Test, std)]
Index: []

[0 rows x 24 columns]

<IPython.core.display.Javascript object>

In [43]:
c

Category     Company  Data Shape Timesteps  \
                                                   
0  Local Model  partner_i  (1234, 42)       NaN   
1  Local Model  partner_i  (1234, 42)       NaN   
2  Local Model  partner_i  (1234, 42)       NaN   
3  Local Model  partner_i  (1234, 42)       NaN   

                                        Features              Model  \
                                                                      
0  Chemical + Mineralogical + Physical + One-Hot  Linear Regression   
1  Chemical + Mineralogical + Physical + One-Hot  Linear Regression   
2  Chemical + Mineralogical + Physical + One-Hot  Linear Regression   
3  Chemical + Mineralogical + Physical + One-Hot  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   0.700636  0.049022  ...   
1                                {"Test Size": 0.2}   0.815584  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   0.855466  0.005967  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   0.738757  0.058813  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.980729  0.001604  0.949926  0.143246  0.733407  0.106088  0.017101   
1  0.973914  0.000000  1.212233  0.000000  0.900702  0.000000  0.020311   
2  0.971171  0.000459  0.965048  0.217141  0.718911  0.041660  0.016602   
3  0.977559  0.002862  1.158192  0.309342  0.807439  0.128733  0.018491   

              R2 Test            
        std      mean       std  
0  0.002711  0.961040  0.015278  
1  0.000000  0.940500  0.000000  
2  0.000845  0.961720  0.020438  
3  0.002644  0.945201  0.026837  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>